# Modelagem de tópicos em textos através do Latent Dirichlet Allocation (LDA) aplicação em python 


## imports 

In [1]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from time import time

[nltk_data] Downloading package stopwords to /home/renato/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('../Dados coletados/todos_sites.csv')

df

,Unnamed: 0,TITLE,SUMMARY
0,0,Confira o 2º dia de desfiles da Festlingerie J...,"<img src=""https://s2.glbimg.com/i-uqwYxqUJOApB..."
1,1,Queimada consumiu 850 hectares do Parque Estad...,"<img src=""https://s2.glbimg.com/o1U8CVISCkqRNp..."
2,2,Polícia investiga suspeita de mortes por inges...,Três pessoas já morreram e outra está hospital...
3,3,Cadela abre buraco em raiz de árvore para ter ...,"<img src=""https://s2.glbimg.com/oZVJrG_qs35bZ8..."
4,4,PE confirma mais quatro casos e totaliza 22 re...,"<img src=""https://s2.glbimg.com/Gv9miVzhCl5Qtn..."
...,...,...,...
15607,1674,"Em aniversário da Revolução Chinesa, Hong Kong...",A polícia de choque de Hong Kong prendeu pelo ...
15608,1675,Ataque do Azerbaijão fere 4 repórteres em terr...,"Quatro jornalistas, dois franceses e dois armê..."
15609,1676,Primeiro país europeu atingido pelo coronavíru...,"Nem os próprios italianos parecem acreditar, m..."
15610,1677,"Bailarinas do Faustão, Regina Duarte e videoga...","Há seis meses, quando <a href=""https://www1.fo..."


In [4]:

# remove special characters and digits
def clean_noticia(noticia):
   
    # remove links e alguns pontos
    noticia = re.sub(r'<img src="https\S+', "", noticia)
    noticia = re.sub(r"http\S+", "", noticia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    
    # remove alguns caracteres
    noticia  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",noticia )
    noticia = ' '.join(s for s in noticia.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    noticia = noticia.replace("\n", "")
    return noticia

In [5]:
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['que', 'até', 'esse', 
                    'essa', 'pro', 'pra',
                    'oi', 'lá', 'blá', 'dos'])
clean_noticias = []
for w in range(len(df.SUMMARY)):
  if df['SUMMARY'].iloc[w] == 'nan' or df['SUMMARY'].iloc[w] == '-' or type(df['SUMMARY'].iloc[w]) !=" <class 'str'>":
    noticia = df['TITLE'].iloc[w]
  else:
    noticia = df['SUMMARY'].iloc[w]
  #print(type(noticia))
  noticia = clean_noticia(noticia)
  #print(noticia)  
  clean_noticias.append(noticia)

#remover colunas repetidas
print(len(clean_noticias))
clean_noticias[:10]


15612


['confira dia desfiles festlingerie juruaia assista vídeo',
 'queimada consumiu hectares parque estadual aguapeí diz fundação',
 'polícia investiga suspeita mortes por ingestão bebida alcoólica riachão dantas',
 'cadela abre buraco raiz árvore para ter filhotes precisam ser resgatados pelos bombeiros aparecida goiânia',
 'confirma mais quatro casos totaliza registros síndrome inflamatória multissistêmica pediátrica',
 'eleições como consultar número título eleitor',
 'tabela elaborada pelo procon sergipe faz comparativo preços cesta básica',
 'vivo assista jornal rondônia edição',
 'agências inss voltam funcionar ubá barbacena são joão del rei',
 'denuncia exprefeito orlândia por usurpação cargo público']

### LDA funciona baseado em frequências de palavras, então usaremos TFs, e não TF-IDFs.

In [6]:
# COUNT vectorizer
tf_vectorizer = CountVectorizer(
        min_df = 30,
        max_df = 0.5,
        max_features = 10000,
        stop_words = stop_words, 
        ngram_range = (1,2)
  )

#transform
vec_text = tf_vectorizer.fit_transform(clean_noticias)

#returns a list of words.
words = tf_vectorizer.get_feature_names()
#print(clean_tweets)

print(len(words))

752


In [7]:
from sklearn.decomposition import LatentDirichletAllocation

In [8]:
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("\n--\nTopic #{}: ".format(topic_idx + 1))
    message = ", ".join([feature_names[i]
                          for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
  print()

def display_topics(W, H, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for d in top_d_idx: 
          doc_data = df[['TITLE', 'SUMMARY']].iloc[d]
          print(f'{doc_data[0]} -  \t{W[d, topic_idx]:.2f}')

In [9]:
lda = LatentDirichletAllocation(n_components=10, 
                                learning_method='online', # 'online' equivale a minibatch no k-means
                                random_state=0)

t0 = time()

lda.fit(vec_text)
doc_topic_matrix = lda.transform(vec_text)

print("done in %0.3fs." % (time() - t0))

done in 17.770s.


In [10]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(15612, 10)
Matriz tópicos-termos:(10, 752)


In [12]:
display_topics(doc_topic_matrix,
               lda.components_, 
               words,
               df,
               15, 
               10)


--
Topic #1: 
PODE, EMERGENCIAL, AUXÍLIO EMERGENCIAL, SER, BOLSA, NOVA, SAIBA, BOLSA FAMÍLIA, FAZ, PREFEITURA, PODE SER, PODEM, CAMPANHA, CADASTRO, MAIOR
Pausa no FIES pode ser solicitada por todos? -  	0.85
Calor excessivo aumenta risco de infarto e derrame, dizem especialistas -  	0.82
Desconfiança e crise sabotam reforma tributária (por Leonardo Barreto) -  	0.82
Valor do IPVA atrasado pode ser parcelado? -  	0.82
IPVA em atraso pode ser pago em até 10x por condutores -  	0.82
IPVA atrasado pode ser parcelado por todos? -  	0.82
Covid-19: avião pode ser fretado para resgatar bebês nascidos de barriga de aluguel na Rússia -  	0.82
Parque dos Falcões poderá ser Patrimônio Cultural de Sergipe -  	0.82
Denúncias em campanhas eleitorais podem ser feitas pelo App Pardal -  	0.82
Conta do Instagram pode ser recuperada? -  	0.82

--
Topic #2: 
APÓS, ANOS, APOSENTADORIA, RIO, CASA, MORRE, RECEBE, PEDE, VAGAS, JOVEM, MOTORISTA, HOJE, PRESIDENTE, HOMEM, ABRE
Motorista morre após caminhão tomb

In [13]:
#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [14]:
pyLDAvis.sklearn.prepare(lda, vec_text, tf_vectorizer, sort_topics=False, mds = 'tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1  12.179681
1       0.648955  71.707100       2        1  10.610272
2      71.019684   9.971006       3        1  11.311983
3       3.785981  33.185135       4        1   9.438297
4     -42.666500  -9.222070       5        1  13.321389
5      -3.621689 -43.526516       6        1   7.295597
6      -3.632402  -4.171708       7        1   8.721656
7     -36.584400  35.804413       8        1   9.670168
8      44.391380  50.615383       9        1   8.950345
9      40.235809 -31.591665      10        1   8.500610, topic_info=                  Term         Freq        Total Category  logprob  loglift
338               inss  1575.000000  1575.000000  Default  30.0000  30.0000
224                diz   744.000000   744.000000  Default  29.0000  29.0000
65             auxílio   630.000000   630.000000  Default  28.0000  28.0000
42                após   752.000000   752.000000  Default  27.0000  27.0000
659              sobre   463.000000   463.000000  Default  26.0000  26.0000
..                 ...          ...          ...      ...      ...      ...
380              marca    50.681631    51.597733  Topic10  -4.6125   2.4471
439  nesta quartafeira    50.070393    50.986477  Topic10  -4.6246   2.4469
42                após   167.378339   752.057592  Topic10  -3.4178   0.9625
646                ser   129.718149   487.949860  Topic10  -3.6727   1.1402
233            durante    59.571157   246.726209  Topic10  -4.4509   1.0439

[362 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2         2  0.988118      abre
3         5  0.983632    acesso
4         7  0.992449  acidente
6         6  0.983284    acordo
11        7  0.991672    agosto
...     ...       ...       ...
743       7  0.991772      zona
744      10  0.977002      água
746       6  0.983887   íntegra
748       7  0.983600    ônibus
751       2  0.994450     único

[345 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])